<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Исследование-моделей-без-учета-дизбаланса" data-toc-modified-id="Исследование-моделей-без-учета-дизбаланса-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей без учета дизбаланса</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений-1.0" data-toc-modified-id="Дерево-решений-1.0-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево решений 1.0</a></span></li><li><span><a href="#Случайный-лес-1.0" data-toc-modified-id="Случайный-лес-1.0-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес 1.0</a></span></li><li><span><a href="#Логистическая-регрессия-1.0." data-toc-modified-id="Логистическая-регрессия-1.0.-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия 1.0.</a></span></li><li><span><a href="#Выводы-по-исследованию-моделей-без-учета-дизбаланса" data-toc-modified-id="Выводы-по-исследованию-моделей-без-учета-дизбаланса-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Выводы по исследованию моделей без учета дизбаланса</a></span></li></ul></li><li><span><a href="#Борьба-с-дизбалансом" data-toc-modified-id="Борьба-с-дизбалансом-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Борьба с дизбалансом</a></span><ul class="toc-item"><li><span><a href="#Взвешивание-классов" data-toc-modified-id="Взвешивание-классов-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Взвешивание классов</a></span></li><li><span><a href="#Upsampling" data-toc-modified-id="Upsampling-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Upsampling</a></span></li><li><span><a href="#Downsampling" data-toc-modified-id="Downsampling-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Downsampling</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import tree

from sklearn.metrics import f1_score #(y_true, y_pred)
from sklearn.metrics import confusion_matrix #(y_true, y_pred)
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
# Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from joblib import dump

In [ ]:
RANDOM_STATE = 12345

In [ ]:
df = pd.read_csv('/datasets/Churn.csv')


## Подготовка данных

In [ ]:
display(df.head())
df.info()

В столбце Tenure (сколько лет человек является клиентом банка) у нас есть NaN значения. Они составляют меньше 10% данных, так что просто отбросим их.

In [ ]:
df = df.dropna()

Фамилии клиентов с большой вероятностью не влияют на их банковскую активность, а в случае применения One hot encoding (прямое кодирование) этот столбец превратится в тысячу лишних переменных. Удалим его, а потом применим One hot encoding ко всей таблице, чтобы преобразовать остальные категориальные признаки в численные. Таких категориальных переменных останется две - страна (Франция, Испания и Германия) и гендер (муж., жен.). 

Так как индекс строки с изначальных данных и идентификационный номер клиента по факту не должны влиять на целевой признак, отбросим их в финальной таблице.

In [ ]:
df = df.drop('Surname', axis = 1)
df_ohe = pd.get_dummies(df, drop_first=True)
df_ohe = df_ohe.drop(['RowNumber', 'CustomerId'], axis = 1)

In [ ]:
display(df_ohe.head())
df_ohe.info()

Мы также видим, что некоторые параметры, например, Balance и EstimatedSalary принимают гораздо большие значения, чем все остальные. Исправить это поможет масштабирование признаков через стандартизацию данных. Перед этим разобьем данные на параметры и целевой признак.

In [ ]:
features = df_ohe.drop(['Exited'], axis=1)
target = df_ohe['Exited']

Разобьем данные на обучающую, валидационную и тестовую выборки.

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(features,target, test_size=0.4, random_state=RANDOM_STATE)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid,target_valid, test_size=0.5, random_state=RANDOM_STATE)

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train) #fitting scaler to train features
features_train = pd.DataFrame(scaler.transform(features_train)) #scaling train features
features_valid = pd.DataFrame(scaler.transform(features_valid)) #scaling valid features
features_test = pd.DataFrame(scaler.transform(features_test)) #scaling test features

In [ ]:
features_train.columns = features.columns
features_valid.columns = features.columns
features_test.columns = features.columns
features_test.head()

## Исследование задачи

В данном случае у нас есть два класса в целевом признаке, то есть мы решаем задачу классификации, а не предсказания. Таким образом, нам здесь понадобятся модели классификаторы: возьмем дерево решений, случайный лес и логистическую регрессию.

В качестве метрик успешности модели возьмем ***F1***, объединяющую в себе точность и полноту (***precision** = true positives/all predicted positive и **recall** (он же true positive rate) = true possitives/all ground truth positives), и area under the curve of ROC (receiver operating characteritic - то есть функция отношения ***true posititive rate*** к ***false positive rate***). 

Чем выше F1 (*= (2 * Precision * Recall) / (Precision + Recall*), тем лучше баланс модели между полнотой и точностью. 

Чем больше область под функцией ROC, тем успешнее модель.

Метрики mean absolute error и mean squared error нам здесь не понадобятся, так они оценивают качество регрессии-предсказателя.

In [ ]:
#Посмотрим на баланс классов 

target_train.sum()/len(target_train)


Мы видим значительный дизбаланс классов в целевом признаке - ответы 1 составляют всего 20% данных. Сперва попробуем обучить модели без учета дизбаланса.

## Исследование моделей без учета дизбаланса

Опробуем три модели классификатора:
    
    - Дерево решений (DecisionTreeClassifier)
    - Случайный лес (RandomForestClassifier)
    - Логистическая регрессия (LogisticRegression)


### Дерево решений 1.0

In [ ]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for depth in range(1, 15):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth) #опробуем глубину дерева от 1 до 15
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid) 
    result = f1_score(target_valid, predictions)
    if result > best_tree_result:
        best_tree_model = model
        best_tree_depth = depth
        best_tree_result = result
print()        
print("F1 score лучшей модели дерева решений на валидационной выборке:", best_tree_result, ", с глубиной", best_tree_depth)

In [ ]:
plt.figure(figsize=(12,12))
tree.plot_tree(best_tree_model,fontsize=10, feature_names = features.columns, class_names = ['Exited', 'Not Exited'])
plt.show()

In [ ]:
best_tree_predictions = best_tree_model.predict(features_valid)

disp = ConfusionMatrixDisplay(confusion_matrix(target_valid, best_tree_predictions))
disp.plot()
plt.show()

### Случайный лес 1.0

In [ ]:
best_forest_model = None
best_forest_result = 0
best_forest_depth = 0
best_forest_est = 0

for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=est,max_depth=depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid) 
        result = f1_score(target_valid, predictions)
        if result > best_forest_result:
            best_forest_model = model
            best_forest_result = result
            best_forest_est = est
            best_forest_depth = depth

print('F1 score лучшей модели случайный лес:',best_forest_result, ",", "с глубиной", best_forest_depth, "и числом деревьев", best_forest_est)

In [ ]:
best_forest_predictions = best_forest_model.predict(features_valid)

disp = ConfusionMatrixDisplay(confusion_matrix(target_valid, best_forest_predictions))
disp.plot()
plt.show()

### Логистическая регрессия 1.0.

In [ ]:
log_model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=1000)
log_model.fit(features_train, target_train)
dump(model, 'model_9_1.joblib')
predictions = log_model.predict(features_valid) 
log_model_result = f1_score(target_valid, predictions)
print('F1 score модели логистическая регрессия:',log_model_result)

In [ ]:
log_predictions = log_model.predict(features_valid)

disp = ConfusionMatrixDisplay(confusion_matrix(target_valid, log_predictions))
disp.plot()
plt.show()

### Выводы по исследованию моделей без учета дизбаланса

- F1 score лучшей модели дерева решений на валидационной выборке: 0.57
- F1 score лучшей модели случайный лес: 0.57
- F1 score модели логистическая регрессия: 0.30

Для всех моделей F1 score не дотягивает до 0.59. Построив матрицу истинно отрицательных, истинно положительных, ложно отрицательных и ложно положительных ответов для каждой модели, мы видим, что дизбаланс классов дейсвительно сказывается на поведении модели - она делает много ошибок в предсказании класса 1 (то есть покинувшие банк клиенты), который нам интересен. 

Попробуем решить проблему дизбаланса.



## Борьба с дизбалансом

Опробуем три метода борьбы с дизбалансом - 
1) взвешивание классов <br>
2) увеличение выборки за счет дублирования редкого класса (upsampling) <br>
3) уменьшение выборки за счет отбрасывание частого класса (downsampling)

### Взвешивание классов



Опробуем те же три модели.

In [ ]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for depth in range(1, 15):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth, class_weight='balanced') #опробуем глубину дерева от 1 до 15
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid) 
    result = f1_score(target_valid, predictions)
    if result > best_tree_result:
        best_tree_model = model
        best_tree_depth = depth
        best_tree_result = result
print()        
print("F1 score лучшей модели дерева решений на валидационной выборке:", best_tree_result, ", с глубиной", best_tree_depth)

In [ ]:
best_forest_model = None
best_forest_result = 0
best_forest_depth = 0
best_forest_est = 0

for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=est,max_depth=depth, class_weight='balanced')
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid) 
        result = f1_score(target_valid, predictions)
        if result > best_forest_result:
            best_forest_model = model
            best_forest_result = result
            best_forest_est = est
            best_forest_depth = depth

print('F1 score лучшей модели случайный лес:',best_forest_result, ",", "с глубиной", best_forest_depth, "и числом деревьев", best_forest_est)

In [ ]:
log_model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=1000, class_weight='balanced')
log_model.fit(features_train, target_train)
dump(model, 'model_9_1.joblib')
predictions = log_model.predict(features_valid) 
log_model_result = f1_score(target_valid, predictions)
print('F1 score модели логистическая регрессия:',log_model_result)

**Выводы по результам моделей с взвешиванием классов:** <br>
    - На F1 score дерева решений взвешивание классов почти не повлияло <br>
    - Взвешивание классов повысило F1 score случайного леса до 0.64, что выше 0.59 на валидационной выборке <br>
    - Взвешивание классов также повысило F1 score логистической регрессии с 0.3 до 0.5!<br>

### Upsampling

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=RANDOM_STATE)
    return features_upsampled, target_upsampled

In [ ]:
features_train.reset_index(drop = True, inplace = True)
target_train.reset_index(drop = True, inplace = True)

In [ ]:
features_upsample, target_upsample = upsample(features_train, target_train, 4)

In [ ]:
target_upsample.sum()/len(target_upsample)

# Теперь баланс классов 1:1

Посмотрим на три наши модели.

In [ ]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for depth in range(1, 15):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth, class_weight='balanced') #опробуем глубину дерева от 1 до 15
    model.fit(features_upsample, target_upsample)
    predictions = model.predict(features_valid) 
    result = f1_score(target_valid, predictions)
    if result > best_tree_result:
        best_tree_model = model
        best_tree_depth = depth
        best_tree_result = result
print()        
print("F1 score лучшей модели дерева решений на валидационной выборке:", best_tree_result, ", с глубиной", best_tree_depth)

In [ ]:
best_forest_model = None
best_forest_result = 0
best_forest_depth = 0
best_forest_est = 0

for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=est,max_depth=depth, class_weight='balanced')
        model.fit(features_upsample, target_upsample)
        predictions = model.predict(features_valid) 
        result = f1_score(target_valid, predictions)
        if result > best_forest_result:
            best_forest_model = model
            best_forest_result = result
            best_forest_est = est
            best_forest_depth = depth

print('F1 score лучшей модели случайный лес:',best_forest_result, ",", "с глубиной", best_forest_depth, "и числом деревьев", best_forest_est)

In [ ]:
log_model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=1000, class_weight='balanced')
log_model.fit(features_upsample, target_upsample)
dump(model, 'model_9_1.joblib')
predictions = log_model.predict(features_valid) 
log_model_result = f1_score(target_valid, predictions)
print('F1 score модели логистическая регрессия:',log_model_result)

-------
**Выводы по результам моделей с взвешиванием классов:** <br>
  Для модели случайного леса F1 score повысился выше нужного нам порога! <br>
- F1 score лучшей модели дерева решений на валидационной выборке: 0.57
- F1 score лучшей модели случайный лес: 0.63
- F1 score модели логистическая регрессия: 0.50

### Downsampling

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
    [features_zeros.sample(frac=fraction, random_state=RANDOM_STATE)] + [features_ones])
    target_downsampled = pd.concat(
    [target_zeros.sample(frac=fraction, random_state=RANDOM_STATE)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=RANDOM_STATE)
    return features_downsampled, target_downsampled

In [ ]:
features_downsample, target_downsample = downsample(features_train, target_train, 1/4)

In [ ]:
target_downsample.sum()/len(target_downsample)

# Теперь баланс классов 1:1


In [ ]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for depth in range(1, 15):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth, class_weight='balanced') #опробуем глубину дерева от 1 до 15
    model.fit(features_downsample, target_downsample)
    predictions = model.predict(features_valid) 
    result = f1_score(target_valid, predictions)
    if result > best_tree_result:
        best_tree_model = model
        best_tree_depth = depth
        best_tree_result = result
print()        
print("F1 score лучшей модели дерева решений на валидационной выборке:", best_tree_result, ", с глубиной", best_tree_depth)

In [ ]:
best_forest_model = None
best_forest_result = 0
best_forest_depth = 0
best_forest_est = 0

for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=est,max_depth=depth, class_weight='balanced')
        model.fit(features_downsample, target_downsample)
        predictions = model.predict(features_valid) 
        result = f1_score(target_valid, predictions)
        if result > best_forest_result:
            best_forest_model = model
            best_forest_result = result
            best_forest_est = est
            best_forest_depth = depth

print('F1 score лучшей модели случайный лес:',best_forest_result, ",", "с глубиной", best_forest_depth, "и числом деревьев", best_forest_est)

In [ ]:
log_model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=1000, class_weight='balanced')
log_model.fit(features_downsample, target_downsample)
dump(model, 'model_9_1.joblib')
predictions = log_model.predict(features_valid) 
log_model_result = f1_score(target_valid, predictions)
print('F1 score модели логистическая регрессия:',log_model_result)

-------
**Выводы по результам моделей с взвешиванием классов:** <br>
  Уменьшение выборки показало результат чуть хуже, чем увеличение выборки, что, наверное, неудивительно, ведь тамким образом мы подаем в модель меньше данных. <br>
- F1 score лучшей модели дерева решений на валидационной выборке: 0.57
- F1 score лучшей модели случайный лес: 0.59
- F1 score модели логистическая регрессия: 0.50


**Вывод по дизбалансу классов:** <br>

Из трех опробованных методов балансирования классов лучший результат показало увеличение выборки (upsampling). Лучший результат при этом показала модель случайного леса с результатом 0.63 на валидационной выборке. Это модель с глубиной 9 и числом деревьев 40.


## Тестирование модели

In [ ]:
final_model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=40,max_depth=9, class_weight='balanced')
final_model.fit(features_upsample, target_upsample)

------
Теперь проведем тестирование.

In [ ]:
predictions_forest = final_model.predict(features_test) 
result_forest = f1_score(target_test, predictions_forest)
print('F1 score лучшей модели случайный лес на тестовой выборке:',result_forest)

------
Исследуем метрику AUC-ROC победившей модели **случайного леса с F1 score 0.594** на тестовой выборке

In [ ]:
roc_auc_score(target_test, predictions_forest)

In [ ]:
probabilities_test = final_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

fpr, tpr, thresholds = roc_curve(target_test, probabilities_one_test)# < напишите код здесь >
plt.figure()
plt.plot(fpr, tpr)

# ROC-кривая случайной модели
plt.plot([0, 1], [0, 1], linestyle='--')

plt.xlim(0,1)
plt.ylim(0,1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()


Наша ROC-кривая близится к идеалу, а ROC-AUC достаточно близко к единице - 0.77

Модель готова!